# Table_of_Contents
Show games from the best in each category:
* [Guesser leaderbord](#Guesser_leaderbord)
* [Answerer leaderbord](#Answerer_leaderbord)
* [Guesser submission_leaderbord](#Guesser_submission_leaderbord)
* [Answerer submission leaderbord](#Answerer_submission_leaderbord)
* [Questions Wordcould](#Questions_wordcould)

In [ ]:
import pandas as pd
from ast import literal_eval
from IPython.display import display, Markdown
import string
import json

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [ ]:
df = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv", converters={col_list:literal_eval for col_list in ["answers", "questions", "guesses"]})
df

In [ ]:
def keyword_guessed(guess: str, keyword:str, alts:list[str]) -> bool:
    # From competition source llm_20_questions.py
    def normalize(s: str) -> str:
        t = str.maketrans("", "", string.punctuation)
        return s.lower().replace("the", "").replace(" ", "").translate(t)

    if normalize(guess) == normalize(keyword):
        return True
    for s in alts:
        if normalize(guess) == normalize(s):
            return True

    return False

In [ ]:
exec(open("/kaggle/input/llm-20-questions/llm_20_questions/keywords.py").read()) # create KEYWORDS_JSON
KEYWORDS_JSON = json.loads(KEYWORDS_JSON)
# needed to apply keyword_guessed 
KEYWORD_TO_ALTS = {words["keyword"]:words["alts"] for category in KEYWORDS_JSON for words in category["words"]}
# KEYWORD_TO_ALTS

In [ ]:
# fix `guessed`. Use competition function instead of isin 
df["guessed"]=df.apply(lambda row: keyword_guessed(row.guesses[-1] if len(row.guesses)>0 else "", row.keyword, KEYWORD_TO_ALTS.get(row.keyword,[])), axis=1)

# games count by team, to normalize
guesser_counts = {k:v for k,v in df.guesser.value_counts().items()}
answerer_counts = {k:v for k,v in df.answerer.value_counts().items()}

winner_df = df.loc[df.guessed]
winner_df

In [ ]:
winner_df.guesser.value_counts().head(10)

In [ ]:
def fmt_team_winrate(team, team_fmt = "{:30} winrate: {:.3%} ({}/{})")->str:
    return team_fmt.format(team[0], team[1][0], team[1][1], team[1][2])

def winrate(win_counts:dict, total_counts:dict)->list:
    ratio_win = {k:(v/total_counts.get(k,1), v, total_counts.get(k)) for k,v in win_counts}
    return sorted(ratio_win.items(), key=lambda item: item[1][0], reverse = True)

In [ ]:
def print_winrate(winner_df=winner_df, df=df):
    nb_total_games = len(df)
    nb_wins = len(winner_df)
    print(fmt_team_winrate(("Winrate total",(nb_wins/nb_total_games,nb_wins,nb_total_games))))

print_winrate()

# Guesser_leaderbord

In [ ]:
guesser_win = winrate(winner_df.guesser.value_counts().items(), guesser_counts)
for team in guesser_win[:10]:
    print(fmt_team_winrate(team))

# Answerer_leaderbord
[Table of Contents](#Table_of_Contents)

In [ ]:
answerer_win = winrate(winner_df.answerer.value_counts().items(), answerer_counts)
for team in answerer_win[:10]:
    print(fmt_team_winrate(team))

## Top guessed keywords:

In [ ]:
winner_df.keyword.value_counts() # france in top, probably because of the baseline example

In [ ]:
episode_agents = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/EpisodeAgents.csv")
episode_agents=episode_agents.groupby(["EpisodeId","SubmissionId"]).first()
episode_agents

In [ ]:
def show_game(row:pd.Series):
    if row.empty:
        return 
    table_format = "| {question} | {answer} | {guess} |"
    history = "\n".join([table_format.format(question="question",answer="answer", guess="guess"), table_format.format(question="---",answer="---", guess="---") ] + [table_format.format(question=question,answer=answer, guess=guess) for answer, question, guess in zip(row.answers, row.questions, row.guesses)])
    guesser_score = episode_agents.loc[(row.game_num, row.guesser_SubmissionId)]
    guesser_score = f"{guesser_score.InitialScore:.2f}->{guesser_score.UpdatedScore:.2f} Confidence={guesser_score.UpdatedConfidence:.2f}" if not guesser_score.empty else ""
    
    answerer_score = episode_agents.loc[(row.game_num, row.answerer_SubmissionId)]
    answerer_score = f"{answerer_score.InitialScore:.2f}->{answerer_score.UpdatedScore:.2f} Confidence={answerer_score.UpdatedConfidence:.2f}" if not answerer_score.empty else ""
    
    return display(Markdown(f"""\
* {row.game_num=}
* Guesser:  **{row.guesser}** score: {guesser_score}
* Answerer: **{row.answerer}** score: {answerer_score}
* Keyword: **{row.keyword}** Category: {row.category} 
* Nb round: {row.nb_round} {row.CreateTime}

{history}
"""))
    
show_game(winner_df.iloc[0]) # this example show that `the answer is: france` did not won the game, need exact keyword or one of alts

In [ ]:
display(Markdown(f"## Best answerer **{fmt_team_winrate(answerer_win[0])}**"))
for i, row in winner_df[winner_df.answerer==answerer_win[0][0]].iterrows():
    show_game(row)

In [ ]:
display(Markdown(f"## Best guesser **{fmt_team_winrate(guesser_win[0])}**"))
for i, row in winner_df[winner_df.guesser==guesser_win[0][0]].iterrows():
    show_game(row)

# Best submissions
Same, per SubmissionId instead of just guesser/answerer.  
The goal is to have a more precise leaderbord, a team could submit multiple agents with different strategies and results.    
With the submissionId we can see if a new submission perform well (no matter how many submissions the team made and how many games they played)


In [ ]:
guesser_sub_counts = {k:v for k,v in df.guesser_SubmissionId.value_counts().items()}
answerer_sub_counts = {k:v for k,v in df.answerer_SubmissionId.value_counts().items()}

guesser_sub_win = winrate(winner_df.guesser_SubmissionId.value_counts().items(), guesser_sub_counts)
answerer_sub_win = winrate(winner_df.answerer_SubmissionId.value_counts().items(), answerer_sub_counts)

subId_to_team = {**winner_df.groupby("answerer_SubmissionId").answerer.first(), **winner_df.groupby("guesser_SubmissionId").guesser.first()}

# Guesser_submission_leaderbord
[Table of Contents](#Table_of_Contents)

In [ ]:
for team in guesser_sub_win[:10]:
    team = (f"{subId_to_team[team[0]]} {team[0]}",(team[1]))
    print(fmt_team_winrate(team))

In [ ]:
display(Markdown(f"## Games from the best guesser agents: SubmissionId={fmt_team_winrate(guesser_sub_win[0])}"))
for i, row in winner_df[winner_df.guesser_SubmissionId==guesser_sub_win[0][0]].iterrows():
    show_game(row)

# Answerer_submission_leaderbord
[Table of Contents](#Table_of_Contents)

In [ ]:
for team in answerer_sub_win[:10]:
    team = (f"{subId_to_team[team[0]]} {team[0]}",(team[1]))
    print(fmt_team_winrate(team))

In [ ]:
display(Markdown(f"## Games from the best answerer agents: {subId_to_team[answerer_sub_win[0][0]]} SubmissionId={fmt_team_winrate(answerer_sub_win[0])}"))
for i, row in winner_df[winner_df.answerer_SubmissionId==answerer_sub_win[0][0]].iterrows():
    show_game(row)

In [ ]:
display(Markdown(f"## Games from the 2nd best answerer agents: {subId_to_team[answerer_sub_win[1][0]]} SubmissionId={fmt_team_winrate(answerer_sub_win[1])}"))
for i, row in winner_df[winner_df.answerer_SubmissionId==answerer_sub_win[1][0]].iterrows():
    show_game(row)

# Games from submissions with the best score
Also count previous submissions, not very useful for now

In [ ]:
_, best_subId = episode_agents.iloc[episode_agents.UpdatedScore.argmax()].name
display(Markdown(f"## Submission with best score: {subId_to_team[best_subId]} SubmissionId={best_subId}"))
for i, row in winner_df[winner_df.answerer_SubmissionId==best_subId].iterrows():
    show_game(row)
    
for i, row in winner_df[winner_df.guesser_SubmissionId==best_subId].iterrows():
    show_game(row)

# Questions_wordcould

In [ ]:
t = str.maketrans("", "", string.punctuation)
questions_flat = [question.lower() for questions in df.questions.values for question in questions]
print("nb unique questions:",len(set(questions_flat)))

In [ ]:
Counter(questions_flat).most_common(50)

In [ ]:
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
stop_words

In [ ]:
%%time
top_q_str = " ".join(question for question, count in Counter(questions_flat).most_common(200))
wordcloud = WordCloud(stopwords=stop_words, background_color="white", width=800, height=400).generate(top_q_str)

# Display the generated image:
plt.figure(figsize=(30,20))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
%%time
stop_words+=["keyword","question","questions","sure","please","answer","answerer","answers","yes"]
wordcloud_words = WordCloud(stopwords=stop_words, background_color="white", width=800, height=400).generate(top_q_str)

# Display the generated image:
plt.figure(figsize=(30,20))
plt.imshow(wordcloud_words, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# tfidf = TfidfVectorizer(max_df=.98, min_df=.01, stop_words=stop_words, use_idf=True, norm=None)
# questions_tfidf = tfidf.fit_transform(questions_flat)
# questions_tfidf

In [ ]:
# tfidf.get_feature_names_out()

In [ ]:
# questions_tfidf = pd.DataFrame(questions_tfidf.todense(),columns=tfidf.get_feature_names_out())
# questions_tfidf

# Most asked questions from winning games

In [ ]:
# questions_win = {question for questions in winner_df.questions.values for question in questions}
# print(len(questions_win))
# questions_win

In [ ]:
all_questions_win = [question.lower() for questions in winner_df.questions.values for question in questions]
len(all_questions_win)

In [ ]:
Counter(all_questions_win).most_common(50)

In [ ]:
%%time
stop_words+=["keyword","question","questions","sure","please","answer","answerer","answers","yes"]
wordcloud_words = WordCloud(stopwords=stop_words, background_color="white", width=800, height=400).generate(" ".join(question for question, count in Counter(all_questions_win).most_common(50)))

# Display the generated image:
plt.figure(figsize=(30,20))
plt.imshow(wordcloud_words, interpolation='bilinear')
plt.axis("off")
plt.show()

# Episodes succesful since the keywords maj

In [ ]:
winner_df_maj = winner_df.loc[winner_df.category.isin(["place","things"])]
winner_df_maj

In [ ]:
winner_df_maj.answerer.value_counts()

In [ ]:
winner_df_maj.guesser.value_counts()